In [1]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.orm import declarative_base
from sqlalchemy.schema import CreateTable
from sqlalchemy.sql.expression import insert
from faker import Faker
import os

In [2]:
RANDOM_SEED = 42
SAVE_DIR = "../sql"

In [3]:
# Create a declarative base class
Base = declarative_base()

# Create a Faker instance
Faker.seed(RANDOM_SEED)
fake = Faker()

In [4]:
# Define the user class
class User(Base):
    __tablename__ = "user"
    id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
    surname = Column(String(255), nullable=False)
    age = Column(Integer, nullable=False)
    address = Column(String(255), nullable=False)
    email = Column(String(255), nullable=False)

    def __repr__(self):
        return f"User({self.name}, {self.surname}, {self.age}, {self.address}, {self.email})"
    
    def to_dict(self):
        user_dict = vars(self).copy()
        user_dict.pop("_sa_instance_state")
        return user_dict

In [5]:
# Create a function to generate a random user
def generate_user():
    name = fake.first_name()
    surname = fake.last_name()
    age = fake.random_int(min=18, max=80)
    address = fake.address()
    email = fake.email()
    return User(name=name, surname=surname, age=age, address=address, email=email)

In [6]:
# Create a function to save the sql code to a file
def save_sql_file(sql, filename, save_dir="./sql"):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    # Save the sql code to a file with the given filename in the sql folder
    with open(os.path.join(save_dir, filename), "w") as f:
        f.write(sql)

In [7]:
# Create a list of 5 users
users = [generate_user() for _ in range(5)]

In [8]:
# Create an engine
engine = create_engine("postgresql:///:memory:")

In [9]:
# Create the sql code for creating the user table
create_table = str(CreateTable(User.__table__).compile(engine))
print(create_table)


CREATE TABLE "user" (
	id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 
	surname VARCHAR(255) NOT NULL, 
	age INTEGER NOT NULL, 
	address VARCHAR(255) NOT NULL, 
	email VARCHAR(255) NOT NULL, 
	PRIMARY KEY (id)
)




In [10]:
# Create the sql code for inserting the users into the table
insert_stmt = insert(User.__table__)\
    .values([u.to_dict() for u in users])\
    .compile(compile_kwargs={"literal_binds": True})
insert_users = str(insert_stmt)
print(insert_users)

INSERT INTO "user" (name, surname, age, address, email) VALUES ('Danielle', 'Johnson', 35, '181 Johnson Course
East William, AK 74064', 'jennifermiles@example.com'), ('Jeffrey', 'Lawrence', 69, '654 Jason Track
Curtisfurt, CT 47553', 'cassandra07@example.net'), ('David', 'Nielsen', 23, '310 Kendra Common Apt. 164
Reidstad, GA 49021', 'jason41@example.net'), ('Cynthia', 'Ferguson', 33, '8350 Lydia Valley Suite 641
New Nancy, MD 28370', 'pfoster@example.com'), ('Douglas', 'Taylor', 65, '969 Cox Dam Suite 101
Lake Ernest, TX 55834', 'cruzcaitlin@example.com')


In [11]:
# Save to sql files
save_sql_file(create_table, "V1_01__create_users_table.sql", SAVE_DIR)
save_sql_file(insert_users, "V1_02__insert_users.sql", SAVE_DIR)